In [ ]:
!pip install -U sagemaker

## Execute evaluation job of summarization task for Claude Model

In [20]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor

region = sagemaker.Session().boto_region_name
role = get_execution_role()
script_processor = ScriptProcessor(
    command=['python3'],
    image_uri='820974724107.dkr.ecr.us-east-1.amazonaws.com/fmeval-bedrock-container:latest',
    role=role, 
    base_job_name="fmeval-anthropic-text-summarization",
    instance_type="ml.m5.xlarge", 
    instance_count=1
)

In [21]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(
    code="code/claude_text_summarization.py",
    # arguments = ["arg1", "arg2"], # Arguments can optionally be specified here
    inputs=[ProcessingInput(source="data/xsum_sample.jsonl", 
                            destination="/opt/ml/processing/input/data/"),
           ],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/output/evaluation")
    ],
    wait=False
)

INFO:sagemaker:Creating processing-job with name fmeval-text-summarization-2024-04-10-07-23-46-002


## execute processing job with S3 files

In [16]:
bucket = sagemaker.Session().default_bucket()
print(bucket)
s3_prefix = "SageMaker_LLM_evaluation_job_package"

sagemaker-us-east-1-820974724107


In [17]:
!aws s3 sync code/ "s3://{bucket}/{s3_prefix}/code/"
!aws s3 sync data/ "s3://{bucket}/{s3_prefix}/data/"

In [18]:
script_processor = ScriptProcessor(
    command=['python3'],
    image_uri='820974724107.dkr.ecr.us-east-1.amazonaws.com/fmeval-bedrock-container:latest',
    role=role, 
    base_job_name="fmeval-text-summarization",
    instance_type="ml.m5.xlarge", 
    instance_count=1
)

In [19]:
script_processor.run(
    code=f"s3://{bucket}/{s3_prefix}/code/preprocessing.py",
    # arguments = ["arg1", "arg2"], # Arguments can optionally be specified here
    inputs=[ProcessingInput(source= f"s3://{bucket}/{s3_prefix}/data/xsum_sample.jsonl", 
                            destination="/opt/ml/processing/input/data/"),
           ],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/output/evaluation")
    ],
)

INFO:sagemaker:Creating processing-job with name fmeval-text-summarization-2024-04-10-05-55-32-212


............................................sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
[Errno 17] File exists: '/opt/ml/processing/output/evaluation'
Could not make directories
#015Downloading builder script:   0%|          | 0.00/7.95k [00:00<?, ?B/s]#015Downloading builder script: 100%|██████████| 7.95k/7.95k [00:00<00:00, 27.5MB/s]
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
#015Downloading builder script:   0%|          | 0.00/6.27k [00:00<?, ?B/s]#015Downloading builder script: 100%|██████████| 6.27k/6.27k [00:00<00:00, 25.7MB/s]
2024-04-10 06:02:56,250#011INFO worker.py:1724 -- Started a local Ray instance.
[nltk_data] Downloading package wordnet

## Execute evaluation job of summarization task for OpenAI model (ChatGPT4)

In [23]:
script_processor = ScriptProcessor(
    command=['python3'],
    image_uri='820974724107.dkr.ecr.us-east-1.amazonaws.com/fmeval-bedrock-container:latest',
    role=role, 
    base_job_name="fmeval-openai-text-summarization",
    instance_type="ml.m5.xlarge", 
    instance_count=1
)

In [ ]:
script_processor.run(
    code="code/openai_text_summarization.py",
    # arguments = ["arg1", "arg2"], # Arguments can optionally be specified here
    inputs=[ProcessingInput(source= f"s3://{bucket}/{s3_prefix}/data/xsum_sample.jsonl", 
                            destination="/opt/ml/processing/input/data/"),
           ],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/output/evaluation")
    ],
)

INFO:sagemaker:Creating processing-job with name fmeval-openai-text-summarization-2024-04-10-07-46-30-335


.............................